# Introduction to using AVID with a basic example:

This example will go over some of the basic concepts to get started with AVID.

Assume the following example data set:
 - Patient1
   - pat1_TP1_MR
   - pat1_TP2_MR
   - pat1_TP1_CT
 - Patient2
   - pat2_TP1_MR1
   - pat2_TP1_MR2
   - pat2_TP2_MR1
   - pat2_TP1_CT
   - pat2_TP2_CT
 - Patient3
   - pat3_TP1_MR

The naming convention `[case]_[timepoint]_[actiontag]` encodes properties that are used in this example.

In AVID, data is usually handled in the form of *Artefacts*. An *Artefact* refers to a specific data entry. It contains relevant metadata such as the patient case, time point, image format, as well as the url to where the corresponding file for this entry is located.<br>
We will assume the artefacts for our data set to have already been generated. You can find them in a .xml-like format under ```examples/output/example.avid```. If you are interested how that was done, take a look at the ```datacrawler.py```.<br>
An exemplary entry looks like this:<br>
```xml
<avid:artefact>
    <avid:property key="case">pat1</avid:property>
    <avid:property key="timePoint">TP1</avid:property>
    <avid:property key="actionTag">CT</avid:property>
    <avid:property key="type">result</avid:property>
    <avid:property key="format">itk</avid:property>
    <avid:property key="url">../data/img/pat1_TP1_CT.txt</avid:property>
    <avid:property key="objective">CT</avid:property>
    <avid:property key="invalid">False</avid:property>
    <avid:property key="id">bbe232b8-5740-11ec-85a6-e9d058c65a83</avid:property>
    <avid:property key="timestamp">1638869608.3330662</avid:property>
</avid:artefact>
```

In [1]:
###############################################################################
# Imports
###############################################################################
import os
import avid.common.workflow as workflow

from avid.actions.pythonAction import PythonUnaryBatchAction, PythonBinaryBatchAction
from avid.selectors import ActionTagSelector, CaseSelector
from avid.linkers import CaseLinker, TimePointLinker, FractionLinker

In [2]:
###############################################################################
# Initialize session with existing Artefacts
###############################################################################
session =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'example.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'example'),
                                expandPaths=True,
                                debug=True)

Now that we have the initial artefacts ready, we want to do something with them. What makes things happen in AVID are so-called *Actions*.  
Actions can do all sorts of things, including fitting, registration, resampling, ...  
We will start with a simple action that calls a specified Python-function for each input-artefact. To specify which artefacts should be used as input to an action we use a *Selector*. For example, let's call the below function for each data entry of patient 1, based on the `case` property.

In [3]:
def my_function(outputs, inputs, **kwargs):
    """
        Simple callable that outputs a sentence including the filename of the input
    """
    inputName = os.path.basename(inputs[0])
    
    with open(outputs[0], "w") as ofile:
        ofile.write(f"Result for file '{inputName}'")

In [4]:
pat1_selector = CaseSelector('pat1')

with session:
    PythonUnaryBatchAction(
        inputSelector=pat1_selector,
        actionTag="basic_example1",
        generateCallable=my_function,
        defaultoutputextension="txt"
    ).do()

2023-09-15 16:11:43,287 [INFO] Starting action: PythonUnaryBatchAction_basic_example1 (UID: 79b67504-9eea-4a2c-b08d-05a5626851fb) ...
2023-09-15 16:11:43,291 [INFO] Starting action: my_function (UID: fe55fca8-ecd0-4bd2-a83b-bf71fe8a6bc3) ...
2023-09-15 16:11:43,292 [INFO] Finished action: my_function (UID: fe55fca8-ecd0-4bd2-a83b-bf71fe8a6bc3) -> SUCCESS
2023-09-15 16:11:43,292 [INFO] Starting action: my_function (UID: dff9431d-af70-4fe9-aa0b-2c6080ec686e) ...
2023-09-15 16:11:43,292 [INFO] Finished action: my_function (UID: dff9431d-af70-4fe9-aa0b-2c6080ec686e) -> SUCCESS
2023-09-15 16:11:43,292 [INFO] Starting action: my_function (UID: 370bc74a-62c1-437f-a389-4ea0369a672b) ...
2023-09-15 16:11:43,292 [INFO] Finished action: my_function (UID: 370bc74a-62c1-437f-a389-4ea0369a672b) -> SUCCESS
2023-09-15 16:11:43,292 [INFO] Starting action: my_function (UID: a1b0d37e-c4b9-4fa2-890e-54c71ee4bad3) ...
2023-09-15 16:11:43,308 [INFO] Finished action: my_function (UID: a1b0d37e-c4b9-4fa2-890e

The results can be found in ```examples/output/example_session/basic_example1/result```. Typically, a new folder will be created for the results of each action.  
For each output-file there is now also an artefact we can use in further actions.  
To easily chain actions together, the *Action Tag* can often be useful. It is a property that is usually determined by the action an artefact was produced by. In our previous action, every resulting artefact has its action tag set to *basic_example1*. If no name is specified, by default it will just be the name of the action (here: *PythonUnaryBatchAction*). For the initial artefacts it can make sense to set the action tag based on a certain role the data entry entails, such as *CT* or *MR* here.

Let's use the output we produced in the previous step and use it in a different action.

In [5]:
def extend_content(outputs, inputs):
    """
        Simple callable that reads the content of an input file and writes a new file, including the previous content
    """
    inputName = os.path.basename(inputs[0])
    
    with open(inputs[0], "r") as ifile:
        content = ifile.read()
    with open(outputs[0], "w") as ofile:
        ofile.write(f"New content based on '{inputName}'\nOriginal content: '{content}'")

In [6]:
example1_selector = ActionTagSelector('basic_example1')

with session:
    PythonUnaryBatchAction(inputSelector=example1_selector, 
                        actionTag="basic_example2", 
                        generateCallable=extend_content,
                        defaultoutputextension="txt"
                        ).do()

2023-09-15 16:11:44,073 [INFO] Starting action: PythonUnaryBatchAction_basic_example2 (UID: 6e5ef0e5-c162-48b1-81fc-42a3670eda70) ...
2023-09-15 16:11:44,088 [INFO] Starting action: extend_content (UID: a898d282-19cb-4978-89e4-717f7001e0e2) ...
2023-09-15 16:11:44,092 [INFO] Finished action: extend_content (UID: a898d282-19cb-4978-89e4-717f7001e0e2) -> SUCCESS
2023-09-15 16:11:44,092 [INFO] Starting action: extend_content (UID: c3099589-5b23-4c73-9833-90dc96ca9067) ...
2023-09-15 16:11:44,092 [INFO] Finished action: extend_content (UID: c3099589-5b23-4c73-9833-90dc96ca9067) -> SUCCESS
2023-09-15 16:11:44,092 [INFO] Starting action: extend_content (UID: 2d75f2ce-a477-4e33-ae57-13f78097c260) ...
2023-09-15 16:11:44,092 [INFO] Finished action: extend_content (UID: 2d75f2ce-a477-4e33-ae57-13f78097c260) -> SUCCESS
2023-09-15 16:11:44,092 [INFO] Starting action: extend_content (UID: 4829f474-abf6-45b9-9ccc-c5a118aba2b7) ...
2023-09-15 16:11:44,092 [INFO] Finished action: extend_content (UID:

Actions can also be given more than a single input, meaning they don't have to work on individual artefacts, but can also work on pairs of artefacts (or even more).  
For example, we could wish to pair up MR images with CT images. We can use ActionTagSelectors to select the desired images, but there is a problem. How do we clarify which artefacts belong together in a pair? Theoretically, each MR image could be paired with each CT image, across patients and time points. To get exactly what we want, there are *Linkers*. In our case, the *CaseLinker* will ensure pairs will only be created between artefacts that share the same case.

In [7]:
def pair_two_images(inputs1, inputs2, outputs):
    """
        Simple callable that outputs the names of the two inputs
    """
    text = f"Matched up two images.  Input 1: {os.path.basename(inputs1[0])}  Input 2: {os.path.basename(inputs2[0])}"
    
    with open(outputs[0], "w") as ofile:
        ofile.write(text)

In [10]:
mr_selector = ActionTagSelector('MR')
ct_selector = ActionTagSelector('CT')

with session:
    PythonBinaryBatchAction(
        inputs1Selector=mr_selector,
        inputs2Selector=ct_selector,
        inputLinker=CaseLinker(),
        actionTag="basic_example3",
        generateCallable=pair_two_images,
        defaultoutputextension="txt"
    ).do()

2023-09-15 16:44:23,217 [INFO] Starting action: PythonBinaryBatchAction_basic_example3 (UID: 289750fd-1e25-4315-ae20-04d6ba920456) ...
2023-09-15 16:44:23,217 [INFO] Starting action: pair_two_images (UID: c5242584-ed3e-40b1-89ab-17661f72c826) ...
2023-09-15 16:44:23,227 [INFO] Finished action: pair_two_images (UID: c5242584-ed3e-40b1-89ab-17661f72c826) -> SUCCESS
2023-09-15 16:44:23,229 [INFO] Starting action: pair_two_images (UID: 5bbce3d6-1155-4e31-aacc-482d807f5294) ...
2023-09-15 16:44:23,229 [INFO] Finished action: pair_two_images (UID: 5bbce3d6-1155-4e31-aacc-482d807f5294) -> SUCCESS
2023-09-15 16:44:23,229 [INFO] Starting action: pair_two_images (UID: e1520c6d-490b-4933-83bf-9eff33990af3) ...
2023-09-15 16:44:23,229 [INFO] Finished action: pair_two_images (UID: e1520c6d-490b-4933-83bf-9eff33990af3) -> SUCCESS
2023-09-15 16:44:23,229 [INFO] Starting action: pair_two_images (UID: 55fce868-9df2-4c32-b5b7-985c60dd1492) ...
2023-09-15 16:44:23,239 [INFO] Finished action: pair_two_ima

Looking into the output folder of *basic_example3*, we can see the pairs that were matched up.  
For pat1, there are two results: pat1_TP1_MR + pat1_TP1_CT and pat1_TP2_MR + pat1_TP1_CT . We can see, that the CT image for timepoint 1 is matched up twice, with the MR images of timepoint 1 and 2. Alternatively, if we wanted to only link pairs with the same case and timepoint, we can combine different linkers like so: `CaseLinker() + TimePointLinker()`

In [11]:
combined_linker = CaseLinker() + TimePointLinker()

with session:
    PythonBinaryBatchAction(
        inputs1Selector=mr_selector,
        inputs2Selector=ct_selector,
        inputLinker=combined_linker,
        actionTag="basic_example4",
        generateCallable=pair_two_images,
        defaultoutputextension="txt"
    ).do()

2023-09-15 16:46:44,930 [INFO] Starting action: PythonBinaryBatchAction_basic_example4 (UID: 5b98881f-f9d3-4672-a8fe-14d27e26fafe) ...
2023-09-15 16:46:44,945 [INFO] Starting action: pair_two_images (UID: 0d6890a8-961d-43a8-9683-38947066947a) ...
2023-09-15 16:46:44,949 [INFO] Finished action: pair_two_images (UID: 0d6890a8-961d-43a8-9683-38947066947a) -> SUCCESS
2023-09-15 16:46:44,950 [INFO] Starting action: pair_two_images (UID: 024bbc9d-bf8c-428f-b9ca-abc3db258061) ...
2023-09-15 16:46:44,950 [INFO] Finished action: pair_two_images (UID: 024bbc9d-bf8c-428f-b9ca-abc3db258061) -> SUCCESS
2023-09-15 16:46:44,950 [INFO] Starting action: pair_two_images (UID: f78a6a15-ac5a-44c2-aeed-ea29d3ab0fed) ...
2023-09-15 16:46:44,950 [INFO] Finished action: pair_two_images (UID: f78a6a15-ac5a-44c2-aeed-ea29d3ab0fed) -> SUCCESS
2023-09-15 16:46:44,950 [INFO] Starting action: pair_two_images (UID: 680dab2a-33e9-4205-9cad-73703332877c) ...
2023-09-15 16:46:44,950 [INFO] Finished action: pair_two_ima